In [5]:
import pandas as pd
import numpy as np
import scipy
import torch

In [1]:
import pandas as pd

# Load the item data
item_data_path = 'data/sushi3-2016/sushi3.idata'
item_columns = ['item_id', 'name', 'style', 'major_group', 'minor_group', 'heaviness', 'consumption_frequency', 'normalized_price', 'sell_frequency']
item_df = pd.read_csv(item_data_path, sep='\t', header=None, names=item_columns)

# Filter the item data to include only the 10 items used in the paper
item_set_A_ids = [0, 1, 2, 3, 4, 6, 7, 8, 26, 29]
#item_set_A_ids=[i for i in range(100)]
item_set_A_df = item_df[item_df['item_id'].isin(item_set_A_ids)]

# Preprocess the item features
categorical_features = ['style', 'major_group', 'minor_group']
numerical_features = ['heaviness', 'consumption_frequency', 'normalized_price', 'sell_frequency']

# Convert categorical features to strings to ensure get_dummies works correctly

item_features = pd.get_dummies(item_set_A_df[categorical_features].astype(str))
item_features = pd.concat([item_features, item_set_A_df[numerical_features]], axis=1)

# Display the preprocessed item features
print("Item features shape:", item_features.shape)

# Load the user data
user_data_path = 'data/sushi3-2016/sushi3.udata'
user_columns = ['user_id', 'gender', 'age', 'total_time', 'prefecture_longest', 'region_longest', 'east_west_longest', 'prefecture_current', 'region_current', 'east_west_current', 'prefecture_diff']
user_df = pd.read_csv(user_data_path, sep='\t', header=None, names=user_columns)

# Preprocess the user features
categorical_features_user = ['gender', 'age', 'prefecture_longest', 'region_longest', 'east_west_longest', 'prefecture_current', 'region_current', 'east_west_current']

# Convert categorical features to strings to ensure get_dummies works correctly
user_df[categorical_features_user] = user_df[categorical_features_user].astype(str)

user_features = pd.get_dummies(user_df[categorical_features_user])

# Display the preprocessed user features
print("User features shape:", user_features.shape)


Item features shape: (10, 16)
User features shape: (5000, 128)


# Carregando o arquivo de preferencias e removendo as duas primeiras colunas de metadados

In [2]:
import pandas as pd

# Load and parse the preference order data manually
preference_data_path = 'data/sushi3-2016/sushi3a.5000.10.order'
with open(preference_data_path, 'r') as file:
    lines = file.readlines()

# Remove the first row which contains metadata
lines = lines[1:]

# Split each line into a list of preferences
preference_data = [line.strip().split() for line in lines]

# Convert to a DataFrame
preference_df = pd.DataFrame(preference_data)

# Convert all values to integers
preference_df = preference_df.astype(int)

# Rename columns for clarity
preference_df.columns = [f'pref_{i}' for i in range(preference_df.shape[1])]

# Display the processed preference data

# Remove the first two columns if they contain metadata
preference_df = preference_df.drop(columns=['pref_0', 'pref_1'])

# Rename columns for clarity
preference_df.columns = [f'pref_{i}' for i in range(preference_df.shape[1])]

# Display the cleaned preference data
print(preference_df.shape)

(5000, 10)


In [ ]:
class GPPrefenceElicitation:
    def __init__(self):
        pass
    
    def fit(self, user_features, item_features, preference_data, max_iter=100):
        self.user_features = user_features
        self.item_features = item_features
        self.preference_data = preference_data
        self.n_users = user_features.shape[0]
        self.n_items = item_features.shape[0]
        self.maximum_hyperparameters()
        user_covariance = self.kernel_covariance_matrix(user_features, user_features, self.sv_t, self.ls_t)
        item_covariance = self.kernel_covariance_matrix(item_features, item_features, self.sv_k, self.ls_k)
        self.covariance_matrix = np.kron(user_covariance, item_covariance)
        self.precision_matrix = np.linalg.inv(self.covariance_matrix)
    
    def grad_log_p_D_given_f(self, f, preference_data):
        grad = np.zeros_like(f)
        
        for preference in preference_data:
            t, k_1, k_2 = preference
            index_1 = t * self.n_users + k_1
            index_2 = t * self.n_users + k_2
            z = (f[index_1] - f[index_2]) / self.noise_sd
            phi_val = scipy.stats.norm.pdf(z)
            Phi_val = scipy.stats.norm.cdf(z)
            ratio = phi_val / (Phi_val * self.noise_sd)
            grad[index_1] += ratio
            grad[index_2] -= ratio
        
        return grad
        
    def hessian_log_p_D_given_f(self, f, peference_data):
        hessian = np.zeros((f.shape[0], f.shape[0]))
        
        for preference in preference_data:
            t, k_1, k_2 = preference
            index_1 = t * self.n_users + k_1
            index_2 = t * self.n_users + k_2
            z = (f[index_1] - f[index_2]) / self.noise_sd
            phi_val = scipy.stats.norm.pdf(z)
            Phi_val = scipy.stats.norm.cdf(z)
            ratio = phi_val / (Phi_val * self.noise_sd)
            
            second_derivative = (z * ratio + ratio**2) / (self.noise_sd ** 2)
            hessian[index_1, index_1] -= second_derivative
            hessian[index_2, index_2] -= second_derivative
            hessian[index_1, index_2] += second_derivative
            hessian[index_2, index_1] += second_derivative
        
        return hessian
    
    def maximum_a_posteriori(self, initial_f, preference_data, max_iter=100):
        f = initial_f
        for i in range(max_iter):
            hessian = self.hessian_log_p_D_given_f(f, preference_data)
            precision_matrix_post = hessian + self.precision_matrix
            grad = self.grad_log_p_D_given_f(f, preference_data)
            hessian_mult_f = hessian @ f
            f = np.linalg.solve(precision_matrix_post, grad + hessian_mult_f)
        
        return f
    
    def se_kernel(self, x, y, sv, ls):
        return sv * np.exp(-0.5 * np.linalg.norm(x - y) ** 2 / ls ** 2)
    
    def kernel_covariance_matrix(self, x, y, sv, ls):
        n_x = x.shape[0]
        n_y = y.shape[0]
        covariance_matrix = np.zeros((n_x, n_y))
        
        for i in range(n_x):
            for j in range(n_y):
                covariance_matrix[i, j] = self.se_kernel(x[i], y[j], sv, ls)
        
        return covariance_matrix

    def maximum_hyperparameters(self, init_sv_t=1, init_ls_t=1, init_sv_k=1, init_ls_k=1, init_noise_sd=0.1, max_iter=100):
        sv_t = torch.tensor(init_sv_t, requires_grad=True)
        ls_t = torch.tensor(init_ls_t, requires_grad=True)
        sv_k = torch.tensor(init_sv_k, requires_grad=True)
        ls_k = torch.tensor(init_ls_k, requires_grad=True)
        self.noise_sd = torch.tensor(init_noise_sd, requires_grad=True)
        optimizer = torch.optim.Adam([sv_t, ls_t, sv_k, ls_k, self.noise_sd])
        identity = torch.eye(self.n_users * self.n_items)
        for i in range(max_iter):
            optimizer.zero_grad()
            user_covariance = self.kernel_covariance_matrix(self.user_features, self.user_features, sv_t, ls_t)
            item_covariance = self.kernel_covariance_matrix(self.item_features, self.item_features, sv_k, ls_k)
            self.covariance_matrix = np.kron(user_covariance, item_covariance)
            self.precision_matrix = torch.inverse(torch.tensor(self.covariance_matrix))
            
            # Remove no_grad()?
            with torch.no_grad():
                f = self.maximum_a_posteriori(np.zeros(self.n_users * self.n_items))
                hessian = self.hessian_log_p_D_given_f(f, self.preference_data)
                grad = self.grad_log_p_D_given_f(f, self.preference_data)
            
            loss = 0.5 * torch.logdet(self.covariance_matrix @ hessian + identity) + 0.5 * f @ self.precision_matrix @ f - grad
            loss.backward()
            optimizer.step()
        
        self.sv_t = sv_t.item()
        self.ls_t = ls_t.item()
        self.sv_k = sv_k.item()
        self.ls_k = ls_k.item()
        self.noise_sd = self.noise_sd.item()
    
    def predictive_mean_and_variance(self, f_hat, hessian, t_features, x_features):
        k_t_star = torch.zeros(self.n_users, 1)
        for i in range(self.n_users):
            k_t_star[i] = self.se_kernel(t_features, self.user_features[i], self.sv_t, self.ls_t)
        
        k_x_star = torch.zeros(self.n_items, 1)
        for i in range(self.n_items):
            k_x_star[i] = self.se_kernel(x_features, self.item_features[i], self.sv_k, self.ls_k)
        
        kernel_t = self.kernel_covariance_matrix(t_features, t_features, self.sv_t, self.ls_t)
        kernel_x = self.kernel_covariance_matrix(x_features, x_features, self.sv_k, self.ls_k)
        Sigma_star = kernel_t * kernel_x
        
        k_star = torch.kron(k_t_star, k_x_star)
        
        predictive_mean = k_star.T @ self.precision_matrix @ f_hat
        predictive_variance = Sigma_star - k_star.T @ torch.inverse(self.covariance_matrix + torch.inverse(hessian)) @ k_star
        
        return predictive_mean, predictive_variance
    
    def joint_predictive_mean_and_variance(self, f_hat, hessian, t_features, x_1_features, x_2_features):
        k_t_star = torch.zeros(self.n_users, 1)
        for i in range(self.n_users):
            k_t_star[i] = self.se_kernel(t_features, self.user_features[i], self.sv_t, self.ls_t)
        
        k_x_star = torch.zeros(self.n_items, 2)
        for i in range(self.n_items):
            k_x_star[i, 0] = self.se_kernel(x_1_features, self.item_features[i], self.sv_k, self.ls_k)
            k_x_star[i, 1] = self.se_kernel(x_2_features, self.item_features[i], self.sv_k, self.ls_k)
        
        kernel_t = self.kernel_covariance_matrix(t_features, t_features, self.sv_t, self.ls_t)
        Sigma_star = torch.zeros(2, 2)
        Sigma_star[0, 0] = kernel_t * self.kernel_covariance_matrix(x_1_features, x_1_features, self.sv_k, self.ls_k)
        Sigma_star[1, 1] = kernel_t * self.kernel_covariance_matrix(x_2_features, x_2_features, self.sv_k, self.ls_k)
        Sigma_star[0, 1] = kernel_t * self.kernel_covariance_matrix(x_1_features, x_2_features, self.sv_k, self.ls_k)
        Sigma_star[1, 0] = kernel_t * self.kernel_covariance_matrix(x_2_features, x_1_features, self.sv_k, self.ls_k)
        
        k_star = torch.kron(k_t_star, k_x_star)
        
        predictive_mean = k_star.T @ self.precision_matrix @ f_hat
        predictive_variance = Sigma_star - k_star.T @ torch.inverse(self.covariance_matrix + torch.inverse(hessian)) @ k_star
        
        return predictive_mean, predictive_variance
    
    def expected_improvement(self, t_features, x_features, f_hat, hessian, f_best=None):
        if f_best is None:
            f_best = f_hat
        
        predictive_mean, predictive_variance = self.predictive_mean_and_variance(f_hat, hessian, t_features, x_features)
        
        pred_sd = torch.sqrt(predictive_variance)
        z_prime = (predictive_mean - f_best) / pred_sd
        
        return pred_sd * (z_prime * scipy.stats.norm.cdf(z_prime) + scipy.stats.norm.pdf(z_prime))
    
    def maximum_expected_improvement(self, t_features, item_features, f_hat, hessian, f_best=None):
        maximum_ei = -np.inf
        
        for x in item_features:
            ei = self.expected_improvement(t_features, x, f_hat, hessian, f_best)
            if ei > maximum_ei:
                maximum_ei = ei
        
        return maximum_ei
    
    def prob_i_over_j(self, t_features, x_1_index, x_2_index, f_hat, hessian):
        predictive_mean, predictive_variance = self.joint_predictive_mean_and_variance(f_hat, hessian, t_features, self.item_features[x_1_index], self.item_features[x_2_index])
        
        numerator = predictive_mean[0] - predictive_mean[1]
        denominator = predictive_variance[0, 0] - predictive_variance[1, 1] - 2 * predictive_variance[0, 1] - self.noise_sd ** 2
        
        return scipy.stats.norm.cdf(numerator / denominator)

    def expected_value_of_information(self, t_index, x_1_index, x_2_index, f_hat, hessian, f_best=None):
        t_features = self.user_features[t_index]
        maximum_ei = self.maximum_expected_improvement(t_features, self.item_features, f_hat, hessian, f_best)
        
        preference_data_1_2 = self.preference_data + [[t_index, x_1_index, x_2_index]]
        f_hat_1_2 = self.maximum_a_posteriori(f_hat, preference_data_1_2)
        hessian_1_2 = self.hessian_log_p_D_given_f(f_hat_1_2, preference_data_1_2)
        
        preference_data_2_1 = self.preference_data + [[t_index, x_2_index, x_1_index]]
        f_hat_2_1 = self.maximum_a_posteriori(f_hat, preference_data_2_1)
        hessian_2_1 = self.hessian_log_p_D_given_f(f_hat_2_1, preference_data_2_1)
        
        mei_1_2 = self.maximum_expected_improvement(t_features, self.item_features, f_hat_1_2, hessian_1_2, f_best)
        mei_2_1 = self.maximum_expected_improvement(t_features, self.item_features, f_hat_2_1, hessian_2_1, f_best)
        
        prob_1_2 = self.prob_i_over_j(t_features, x_1_index, x_2_index, f_hat, hessian)
        prob_2_1 = 1 - prob_1_2
        
        return -maximum_ei + prob_1_2 * mei_1_2 + prob_2_1 * mei_2_1

    def preference_elicitation(self, t_index, num_queries):
        # Make a copy
        pref_data = self.preference_data.copy()
        f_hat = torch.zeros(self.n_users * self.n_items)
        
        for i in range(num_queries):
            f_hat = self.maximum_a_posteriori(f_hat, pref_data)
            hessian = self.hessian_log_p_D_given_f(f_hat, pref_data)
            
            max_evoi = -np.inf
            for x_1 in range(self.n_items):
                for x_2 in range(x_1 + 1, self.n_items):
                    evoi = self.expected_value_of_information(t_index, x_1, x_2, f_hat, hessian)
                    if evoi > max_evoi:
                        max_evoi = evoi
                        max_x_1 = x_1
                        max_x_2 = x_2
            
            pref_data.append([t_index, max_x_1, max_x_2])